<a href="https://colab.research.google.com/github/Marchinski/AI_Models/blob/main/VGG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pickle
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm

def show_train_hist(hist, show=False, save=False, path='Train_hist.png'):
    """Loss tracker
    
    Plot the losses of generator and discriminator independently to see the trend
    
    Arguments:
        hist {[dict]} -- Tracking variables
    
    Keyword Arguments:
        show {bool} -- If to display the figure (default: {False})
        save {bool} -- If to store the figure (default: {False})
        path {str} -- path to store the figure (default: {'Train_hist.png'})
    """
    x = range(len(hist['losses']))

    y1 = hist['losses']

    plt.plot(x, y1, label='loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

if __name__ == '__main__':
    # initialise the device for training, if gpu is available, device = 'cuda', else: device = 'cpu'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)

    data_dir = './MNIST_data/'
    save_dir = './Results/'
    # create folder if not exist
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    # training parameters
    batch_size = 256
    learning_rate = 0.01
    epochs = 20

    # construct the dataset and data loader
    transform = transforms.Compose([transforms.Resize(64),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=(0.5,), std=(0.5,))])
    train_data = datasets.MNIST(root=data_dir, train=True, transform=transform, download=True)
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

    VGG_net = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=False)
    VGG_net = VGG_net.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(VGG_net.parameters(), lr=learning_rate, momentum=0.9)

    # tracking variables
    train_hist = {}
    train_hist['losses'] = []
    train_hist['per_epoch_ptimes'] = []
    train_hist['total_ptime'] = []

    start_time = time.time()
    for epoch in range(epochs):
        VGG_net.train()
        Loss = []
        epoch_start_time = time.time()
        for (image, labels) in tqdm(train_loader):
            image = image.repeat(1, 3, 1, 1)
            image = image.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = VGG_net(image)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            Loss.append(loss.item())

        epoch_loss = np.mean(Loss)  # mean generator loss for the epoch
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time

        print("Epoch %d of %d with %.2f s" % (epoch + 1, epochs, per_epoch_ptime))
        print("Loss: %.8f" % (epoch_loss))

        # record the loss for every epoch
        train_hist['losses'].append(epoch_loss)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    print('Finished Training')

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))

    show_train_hist(train_hist, save=True, path=save_dir + '/MNIST_GAN_train_hist.png')

cuda


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
  0%|          | 0/235 [00:00<?, ?it/s]

Epoch 1 of 20 with 75.88 s
Loss: 3.06002764


  0%|          | 0/235 [00:00<?, ?it/s]

Epoch 2 of 20 with 75.95 s
Loss: 2.32684458


...

  0%|          | 0/235 [00:00<?, ?it/s]

Epoch 19 of 20 with 75.52 s
Loss: 0.00567625


100%|██████████| 235/235 [01:15<00:00,  3.11it/s]

Epoch 20 of 20 with 75.55 s
Loss: 0.00429075
Finished Training
Avg per epoch ptime: 75.73, total 20 epochs ptime: 1514.56


NameError: ignored